In [2]:
# %run /Users/denis/Documents/Projects/scripts/init.ipy
%run /Users/maayanlab/Documents/init.ipy
os.chdir('../../')

/Users/maayanlab/Documents/Computation/my-venv/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Need help getting started? Try the cookbook for R:
http://www.cookbook-r.com/Graphs/

  warnings.warn(x, RRuntimeWarning)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Infile

In [3]:
infile = 's1-data.dir/HMS_Datasets-merged_filtered.txt'

### Computation

In [4]:
# Get expression data
expression_dataframe = pd.read_table(infile)
expression_dataframe.head()

,DrugName,Conc,well,ERK-1,Cyclin-D1,FOXO3a,PCNA,MitoTracker Green,KI-67,EGFR,...,DAPI_0005,p53,p21,p27Kip1,S6,H2A.X,Cyclin-E1,HCS CellMask Deep Red,fraction,timepoint
0,Dasatinib,0.1,B02,243.525,185.080,381.770,259.440,2223.205,304.040,877.445,...,778.380,331.180,144.830,159.245,829.340,130.130,110.030,1752.890,cytosolic,24h
1,Dasatinib,0.1,B02,238.844,242.560,425.541,320.890,2404.674,202.830,173.193,...,835.101,263.454,147.688,224.404,171.252,99.000,167.349,2719.784,cytosolic,24h
2,Dasatinib,0.1,B02,481.327,294.827,509.228,437.327,5545.748,374.733,1657.381,...,1123.842,431.223,160.381,169.168,1194.530,129.104,135.401,4682.267,cytosolic,24h
3,Dasatinib,0.1,B02,406.365,429.595,805.340,1831.930,5418.895,2579.835,3775.655,...,1446.610,906.585,178.665,296.690,3860.970,169.585,170.150,7472.670,cytosolic,24h
4,Dasatinib,0.1,B02,384.828,342.138,619.287,501.201,7016.862,326.063,1271.431,...,1558.218,569.937,196.937,215.598,993.557,154.293,181.690,3883.621,cytosolic,24h


##### Drugs

In [5]:
drug_dataframe = expression_dataframe[['DrugName', 'Conc', 'timepoint']].drop_duplicates().query('DrugName!="DMSO"').groupby('DrugName').size().rename('len').to_frame()#.to_dict(orient='records')
drug_dataframe.index.name = 'label'
drug_dataframe.reset_index(inplace=True)
drug_dataframe['id'] = drug_dataframe['label']
drug_dataframe['color'] = ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5']
drug_dataframe.sort_values('label', inplace=True)
drug_dataframe

,label,len,id,color
0,Alpelisib,21,Alpelisib,#8dd3c7
1,Dasatinib,21,Dasatinib,#ffffb3
2,Etoposide,21,Etoposide,#bebada
3,Neratinib,21,Neratinib,#fb8072
4,Paclitaxel,21,Paclitaxel,#80b1d3
5,Palbociclib,21,Palbociclib,#fdb462
6,Trametinib,21,Trametinib,#b3de69
7,Vorinostat,21,Vorinostat,#fccde5


In [6]:
with open('../mcf10a-cycif-browser/data/drugs.json', 'w') as openfile:
    openfile.write(json.dumps(drug_dataframe.to_dict(orient='records'), indent=4, separators=(',', ': ')))

##### CSV

In [7]:
infiles = glob.glob('s2-differential_expression.dir/*')

In [41]:
for infile in infiles:
    differential_expression_dataframe = pd.read_table(infile)
    for fraction in ['cytosolic', 'nuclear']:
        fraction_dataframe = differential_expression_dataframe[differential_expression_dataframe['fraction'] == fraction].sort_values(['drug_name', 'timepoint', 'concentration']).reset_index(drop=True)
        fraction_dataframe['start'] = fraction_dataframe.index%21
        fraction_dataframe['end'] = fraction_dataframe['start']+1
        fraction_dataframe['timepoint'] = [int(x.replace('h', '')) for x in fraction_dataframe['timepoint']]
        fraction_dataframe['bonferroni_pvalue'] = ['%.2E' % x if x < 1 else 1 for x in fraction_dataframe['bonferroni_pvalue']]
        fraction_dataframe.to_csv(infile.replace('s2-differential_expression.dir/', '../mcf10a-cycif-browser/data/differential_expression/').replace('.txt', '_'+fraction+'.txt'))

In [42]:
fraction_dataframe.head()

,concentration,drug_name,fraction,logfc,pvalue,statistic,timepoint,bonferroni_pvalue,start,end
0,0.010000,Alpelisib,nuclear,-0.212632,1.038483e-14,2857520.0,24,3.49E-12,0,1
1,0.031623,Alpelisib,nuclear,0.022049,1.912556e-02,3049263.5,24,1,1,2
2,0.100000,Alpelisib,nuclear,-0.322848,9.742399e-43,2236514.5,24,3.27E-40,2,3
3,0.316230,Alpelisib,nuclear,-0.170006,1.396978e-11,2465739.0,24,4.69E-09,3,4
4,1.000000,Alpelisib,nuclear,0.023909,3.799993e-01,2568016.5,24,1,4,5


##### Genes

In [11]:
genes = [x for x in expression_dataframe.columns if x not in ['DrugName', 'Conc', 'well', 'fraction', 'timepoint'] and 'DAPI' not in x and 'CellMask' not in x and 'MitoTracker' not in x]
with open('../mcf10a-cycif-browser/data/genes.json', 'w') as openfile:
    openfile.write(json.dumps(genes))